In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [3]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, 0]
  ng_refs = [2, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(10, 250), (600, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.9, 1.7), (1.4, 1.4, 1.4, 1.55, 1.6, 1.66)]
  wgt = [(0.7, 0.7), (0.7, 0.5, 0.5, 0.5, 0.5, 0.5)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.000306                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.000715
0.000750      2.0  0.000338                  2.0  0.000739
0.001052      3.0  0.000408                  3.0  0.000810
0.001476      4.0  0.000485                  4.0  0.000879
0.002070      5.0  0.000564                  5.0  0.000949
0.002904      6.0  0.000645                  6.0  0.001021
0.004074      7.0  0.000728                  7.0  0.001095
0.005714      8.0  0.000814                  8.0  0.001170
0.008015      9.0  0.000900                  9.0  0.001247
0.011243     10.0  0.000988                 10.0  0.001326
0.015771     11.0  0.001076                 11.0  0.001407
0.022122     12.0  0.001156                 12.0  0.001490
0.031031     13.0  0.001237                 13.0  0.001573
0.043528     14.0  0.001319                 14.0  0.001663
0.061057     15.0  0.001612                 15.0  0.001946
0.085645     16.0  0.002570                 16.0  0.002822
0.120136     17.0  0.004293                 17.0  0.004367
0.168516     18.0  0.006547                 18.0  0.006381
0.236378     19.0  0.009388                 19.0  0.008925
0.331549     20.0  0.012901                 20.0  0.012092
0.465100     21.0  0.017175                 21.0  0.016007
0.652400     22.0  0.022282                 22.0  0.020864
0.915100     23.0  0.026755                 23.0  0.025298
1.283650     24.0  0.026096                 24.0  0.025376
1.800600     25.0  0.021005                 25.0  0.021415
2.525700     26.0  0.015922                 26.0  0.016975
3.542800     27.0  0.011888                 27.0  0.013046
4.969550     28.0  0.008907                 28.0  0.009806
6.970850     29.0  0.006844                 29.0  0.007308
9.778100     30.0  0.005414                 30.0  0.005433
13.715850    31.0  0.004315                 31.0  0.003991
19.239350    32.0  0.003428                 32.0  0.002901
26.987250    33.0  0.002829                 33.0  0.002197
37.855300    34.0  0.002450                 34.0  0.001774
53.100050    35.0  0.002076                 35.0  0.001427
73.887500    36.0  0.001701                 36.0  0.001139
97.662500    37.0  0.001497                 37.0  0.001014
121.437500   38.0  0.001433                 38.0  0.000999
145.212500   39.0  0.001360                 39.0  0.000977
168.987500   40.0  0.001441                 40.0  0.001074
192.762500   41.0  0.002803                 41.0  0.002145
216.537500   42.0  0.009696                 42.0  0.007783
240.312500   43.0  0.023715                 43.0  0.020301
264.087500   44.0  0.043727                 44.0  0.038804
287.862500   45.0  0.066666                 45.0  0.059695
311.637500   46.0  0.079492                 46.0  0.073281
335.412500   47.0  0.091192                 47.0  0.086256
359.187500   48.0  0.105893                 48.0  0.099103
382.962500   49.0  0.122974                 49.0  0.111929
406.737500   50.0  0.142048                 50.0  0.127860
430.512500   51.0  0.162786                 51.0  0.149037
454.287500   52.0  0.181591                 52.0  0.170693
478.062500   53.0  0.200566                 53.0  0.192049
501.837500   54.0  0.221155                 54.0  0.213106
525.612500   55.0  0.243002                 55.0  0.234896
549.387500   56.0  0.265772                 56.0  0.257510
573.162500   57.0  0.289409                 57.0  0.282150
596.937500   58.0  0.326749                 58.0  0.322111
620.712500   59.0  0.362545                 59.0  0.362355
644.487500   60.0  0.392965                 60.0  0.398424
668.262500   61.0  0.417643                 61.0  0.428112
692.037500   62.0  0.435872                 62.0  0.449721
715.812500   63.0  0.447775                 63.0  0.464551
739.587500   64.0  0.453597                 64.0  0.474444
76

# Fluxes by Layer

CRD                              CLIRAD             \
                         flug          fldg      fnetg       flug       fldg   
pressure     level                                                             
0.000000e+00 1     -29.471371  0.000000e+00 -29.471371        NaN        NaN   
1.000000e-08 1            NaN           NaN        NaN -29.569379   0.000002   
6.244000e-04 2     -29.471371  2.544422e-07 -29.471371 -29.569379   0.000002   
8.759000e-04 3     -29.471371  3.578625e-07 -29.471371 -29.569380   0.000002   
1.228600e-03 4     -29.471371  5.057844e-07 -29.471371 -29.569380   0.000002   
1.723400e-03 5     -29.471371  7.176776e-07 -29.471371 -29.569380   0.000002   
2.417400e-03 6     -29.471372  1.021134e-06 -29.471371 -29.569380   0.000003   
3.390900e-03 7     -29.471372  1.455738e-06 -29.471370 -29.569380   0.000003   
4.756500e-03 8     -29.471372  2.078138e-06 -29.471370 -29.569380   0.000003   
6.672000e-03 9     -29.471373  2.969378e-06 -29.471370 -29.569381   0.000004   
9.358900e-03 10    -29.471374  4.245551e-06 -29.471370 -29.569382   0.000005   
1.312780e-02 11    -29.471375  6.072794e-06 -29.471369 -29.569382   0.000007   
1.841450e-02 12    -29.471377  8.688333e-06 -29.471369 -29.569384   0.000009   
2.583020e-02 13    -29.471380  1.242354e-05 -29.471368 -29.569385   0.000012   
3.623230e-02 14    -29.471384  1.775887e-05 -29.471366 -29.569388   0.000016   
5.082340e-02 15    -29.471389  2.538101e-05 -29.471364 -29.569391   0.000023   
7.129060e-02 16    -29.471397  3.672006e-05 -29.471360 -29.569396   0.000032   
1.000000e-01 17    -29.471407  5.541797e-05 -29.471351 -29.569402   0.000047   
1.402710e-01 18    -29.471419  8.823484e-05 -29.471331 -29.569408   0.000074   
1.967600e-01 19    -29.471433  1.456008e-04 -29.471287 -29.569412   0.000122   
2.759970e-01 20    -29.471444  2.448911e-04 -29.471199 -29.569411   0.000204   
3.871000e-01 21    -29.471444  4.147570e-04 -29.471029 -29.569394   0.000346   
5.431000e-01 22    -29.471414  7.019209e-04 -29.470712 -29.569343   0.000591   
7.617000e-01 23    -29.471313  1.178362e-03 -29.470135 -29.569219   0.001007   
1.068500e+00 24    -29.471083  1.920968e-03 -29.469162 -29.568971   0.001678   
1.498800e+00 25    -29.470724  2.891462e-03 -29.467832 -29.568598   0.002600   
2.102400e+00 26    -29.470297  3.966982e-03 -29.466330 -29.568157   0.003689   
2.949000e+00 27    -29.469832  5.098360e-03 -29.464734 -29.567671   0.004906   
4.136600e+00 28    -29.469339  6.278173e-03 -29.463061 -29.567152   0.006223   
5.802500e+00 29    -29.468830  7.526031e-03 -29.461304 -29.566614   0.007620   
8.139200e+00 30    -29.468309  8.899444e-03 -29.459409 -29.566066   0.009095   
1.141700e+01 31    -29.467792  1.048451e-02 -29.457307 -29.565531   0.010669   
1.601470e+01 32    -29.467332  1.237540e-02 -29.454957 -29.565070   0.012383   
2.246400e+01 33    -29.467029  1.469063e-02 -29.452338 -29.564788   0.014317   
3.151050e+01 34    -29.466973  1.766612e-02 -29.449306 -29.564784   0.016668   
4.420010e+01 35    -29.467279  2.165503e-02 -29.445624 -29.565184   0.019736   
6.200000e+01 36    -29.468242  2.699556e-02 -29.441246 -29.566265   0.023826   
8.577500e+01 37    -29.470254  3.379793e-02 -29.436456 -29.568337   0.029106   
1.095500e+02 38    -29.472795  4.055539e-02 -29.432240 -29.570824   0.034449   
1.333250e+02 39    -29.475685  4.748216e-02 -29.428203 -29.573558   0.039996   
1.571000e+02 40    -29.478906  5.453246e-02 -29.424374 -29.576494   0.045685   
1.808750e+02 41    -29.482499  6.218342e-02 -29.420315 -29.579681   0.051898   
2.046500e+02 42    -29.487828  7.540762e-02 -29.412421 -29.584385   0.062644   
2.284250e+02 43    -29.500190  1.150761e-01 -29.385114 -29.595566   0.095748   
2.522000e+02 44    -29.523995  2.056696e-01 -29.318326 -29.617576   0.174939   
2.759750e+02 45    -29.564609  3.694335e-01 -29.195175 -29.655779   0.322438   
2.997500e+02 46    -29.628513  6.210902e-01 -29.007422 -29.716639   0.551435   
3.235250e+02 47    -29.709

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')